In [7]:
from riotwatcher import LolWatcher, ApiError
import pandas as pd
import time
import datetime

#### Conversion unix timestamp

In [13]:
def get_date(unix_time):
    date_time = datetime.datetime.fromtimestamp(unix_time/100)
    date=date_time.strftime('%Y-%m-%d %H:%M:%S')
    return date

In [14]:
unix_time = 1680537773024
get_date(unix_time)

'2502-07-17 18:28:50'

### Using Riot API key

In [15]:
#Complete with your own API key 
api_key = 'RGAPI-xxxx'
watcher = LolWatcher(api_key)
my_region = 'euw1'
#["na1", "euw1", "eun1", "kr", "br1", "jp1", "ru", "oc1", "tr1", "la1", "la2"]
my_summoner='MamaYumYum'

#### Profile information

In [17]:
my_profile = watcher.summoner.by_name(my_region,my_summoner)

In [18]:
print(my_profile)

{'id': 'tfQ1hZABPAxKvX6rkgpqWE7UszeqS8sq7lieyKopKUMu3Z0', 'accountId': 'RiVjJkjVb_T-e_hgpmXYL6oLTO6G8uGjre8oOQM7-sdaSQ', 'puuid': 'La6KBkKxEZGOS-jBVIqKsS8PzTvxz7BP_k_AIeK-EkbUrCgHgGy3UW1XGX4h6qN4S7mnIqwIzCKfYw', 'name': 'MamaYumYum', 'profileIconId': 5622, 'revisionDate': 1680644142223, 'summonerLevel': 524}


In [19]:
my_ranked_stats = watcher.league.by_summoner(my_region,my_profile['id'])
print(my_ranked_stats)

[{'leagueId': '50a6cc74-60c4-4896-aa2a-54c73299f7c7', 'queueType': 'RANKED_SOLO_5x5', 'tier': 'PLATINUM', 'rank': 'IV', 'summonerId': 'tfQ1hZABPAxKvX6rkgpqWE7UszeqS8sq7lieyKopKUMu3Z0', 'summonerName': 'MamaYumYum', 'leaguePoints': 47, 'wins': 102, 'losses': 99, 'veteran': False, 'inactive': False, 'freshBlood': False, 'hotStreak': False}]


### Match information

In [26]:
def get_matches_list(my_region,my_summoner,nbMatches):
    profile=watcher.summoner.by_name(my_region,my_summoner)
    matches_list=watcher.match.matchlist_by_puuid(my_region,profile['puuid'],count=nbMatches)
    return matches_list
    
    

In [31]:
matches_list=get_matches_list(my_region,my_summoner,20)

In [55]:
matches_list

['EUW1_6346829954',
 'EUW1_6346406135',
 'EUW1_6346377480',
 'EUW1_6345492861',
 'EUW1_6344920335',
 'EUW1_6344828393',
 'EUW1_6344780310',
 'EUW1_6340799573',
 'EUW1_6340749762',
 'EUW1_6335975261',
 'EUW1_6335029842',
 'EUW1_6334965764',
 'EUW1_6333590234',
 'EUW1_6333492894',
 'EUW1_6328709921',
 'EUW1_6328662160',
 'EUW1_6328212297',
 'EUW1_6323942872',
 'EUW1_6322470530',
 'EUW1_6322308414']

In [33]:
#The match.by_id function returns a JSON containing all the information about a game
def get_match(my_region,match_id):
    match=watcher.match.by_id(my_region,match_id)
    return match

In [36]:
#We work with the first match of the list
match=get_match(my_region,matches_list[0])


#### Getting list of summoners's puuid in the match using metadata

In [37]:
match['metadata']['participants']

['La6KBkKxEZGOS-jBVIqKsS8PzTvxz7BP_k_AIeK-EkbUrCgHgGy3UW1XGX4h6qN4S7mnIqwIzCKfYw',
 'XHO77Tu6kS8doG9PTTKpwE2qabXAl9U7zVz5WeOQ-vSDboIb1TueOw2FbbhKOMhnZIdxphfqm5vZaA',
 'Bk8A3yDI4mUKGgXvAUz8gnFXTbFHjezYlSap4E6i_eYCI96b0h5XmZdaM6L-8s3l76MizQ60SZIy4A',
 'zTYbdJMJsIhJcOZ-7zm2zhGbaNvXhs9cNpN94B34i8o6xJOaeFiW4mCFWghCqBXWVGh1gfRGPRbJQA',
 'X5RsLcN-33GkO4GyMaAUMXu1xz5TZCv-XoO4KAnMlt5anhBl3pdPF2SDulKg_0ZdjvnKDnXe2CfeqA',
 'EpT4wWWOF1vhn_mfpa9prf_Hv7GUbKRBPvu0KyhgxwM-6MDi4m3CX0mfNKb7NbeqtolPEXGFgq0XOA',
 'Ml0258N-iQC_-Hjz3nsrTBYSfuDzGuBgYxf6cd37ezqQpwuhcnfU4e57fnZ1i12K_j1c6yPE9owlnw',
 '8Kc00AFQ8-DSrNQCrkVui2gYGfJEbVYWPt1LI6gMl1JfYmIuRefN3jn4UxQ6Mmzx6gjR3JFhdqVgFg',
 '0cDp43ui2J4SsP51C-8dOUMTjsXNQ_DeW1yyWw9v6bqgPAJU_5Wg7TgPA9PBnnH70yYKXcemObnUng',
 'z54HaauybF0p4ZOeETOY_DQEien2GjKL31k491Ph3z4YccWIYF6-oLURzMfhUxEdSRRk-xKopsu3jg']

In [39]:
summoner=watcher.summoner.by_puuid(my_region,match['metadata']['participants'][2])
summoner

{'id': '2yWfVNJBjfnX7cP44HMRIbk-y4ERMgyZR8OaH4bitgs5BhM',
 'accountId': 'JhYLPxAvbDjC22cJMKdlGWI1rOHBtuzfKjAAyGvL1dBzLg',
 'puuid': 'Bk8A3yDI4mUKGgXvAUz8gnFXTbFHjezYlSap4E6i_eYCI96b0h5XmZdaM6L-8s3l76MizQ60SZIy4A',
 'name': 'H4riis',
 'profileIconId': 1296,
 'revisionDate': 1680644142223,
 'summonerLevel': 165}

In [40]:
def get_list_participants(my_region,match_id):
    match=get_match(my_region,match_id)
    list_participants=[]
    for element in match['metadata']['participants']:
        profile=watcher.summoner.by_puuid(my_region,element)
        list_participants.append(profile['name'])
    return list_participants

In [42]:
list_participants=get_list_participants(my_region,matches_list[0])
list_participants

['MamaYumYum',
 'DreamAboutMe',
 'H4riis',
 'Mythic V2',
 'lce Wallow Come',
 'Solarys IV',
 'MMT Darkyyy',
 'ScottTheUnit',
 'Baldulf',
 'Guv']

#### Using info

In [43]:
match['info']['participants'][0]['summonerName']

'MamaYumYum'

In [44]:
summoner_name=watcher.summoner.by_puuid(my_region,match['info']['participants'][0]['puuid'])
summoner_name

{'id': 'tfQ1hZABPAxKvX6rkgpqWE7UszeqS8sq7lieyKopKUMu3Z0',
 'accountId': 'RiVjJkjVb_T-e_hgpmXYL6oLTO6G8uGjre8oOQM7-sdaSQ',
 'puuid': 'La6KBkKxEZGOS-jBVIqKsS8PzTvxz7BP_k_AIeK-EkbUrCgHgGy3UW1XGX4h6qN4S7mnIqwIzCKfYw',
 'name': 'MamaYumYum',
 'profileIconId': 5622,
 'revisionDate': 1680644142223,
 'summonerLevel': 524}

### Game Details

In [59]:
def game_resume(my_region,match_id):
    game_info_df=[]
    match=get_match(my_region,match_id)
    game_info=match['info']
    
    for row in game_info['participants']:
        participants_row={}
        participants_row['summonerName']=row['summonerName']
        if row['individualPosition']=="Invalid":
            participants_row['lane']='ARAM'
        else:  
            participants_row['lane']=row['individualPosition']
        participants_row['champion']=row['championName']
        participants_row['kills']=row['kills']
        participants_row['assists']=row['assists']
        participants_row['deaths']=row['deaths']
        participants_row['kda']=round((row['kills']+row['assists'])/row['deaths']
                                       ,2)
        participants_row['total damages']=row['totalDamageDealtToChampions']
        participants_row['win']=row['win']
        game_info_df.append(participants_row)
    df=pd.DataFrame(game_info_df)
    return df
    


In [60]:
game_resume=game_resume(my_region,matches_list[5])
game_resume

,summonerName,lane,champion,kills,assists,deaths,kda,total damages,win
0,Atomare,TOP,Malphite,2,6,2,4.00,15220,True
1,Jaegha,JUNGLE,Amumu,3,7,3,3.33,11208,True
2,MamaYumYum,MIDDLE,Zed,7,5,4,3.00,13462,True
3,BMO0333,BOTTOM,Twitch,21,6,3,9.00,33212,True
4,ShadowPlays66,UTILITY,Seraphine,4,21,4,6.25,16002,True
5,Gyref,TOP,Chogath,3,2,4,1.25,13835,False
6,Corndawgfan,JUNGLE,MonkeyKing,1,5,8,0.75,9932,False
7,Μars,MIDDLE,Annie,6,5,6,1.83,15296,False
8,reeBro,BOTTOM,Zeri,2,3,7,0.71,4837,False
9,srikig,UTILITY,Senna,4,8,12,1.00,7311,False


### Champions details

In [61]:
#Building a list with the last played champs
last_champ_played=[]
for match in my_matches:
    match_played=watcher.match.by_id(my_region,match)
    for participants in match_played['info']['participants']:
        if participants['summonerName']=="MamaYumYum":
            last_champ_played.append(participants['championName'])
            break
        

In [62]:
last_champ_played

['Singed',
 'Kassadin',
 'Cassiopeia',
 'Vi',
 'AurelionSol',
 'Zed',
 'Yone',
 'Chogath',
 'Zed',
 'Gragas',
 'Neeko',
 'Sylas',
 'Yasuo',
 'Jayce',
 'Shyvana',
 'Fiora',
 'Taric',
 'Gragas',
 'Twitch',
 'Irelia']

### Filtering the games
We only want ranked or ranked flex games. We get rid of the other games with the attribute 'queueId'.  
400 : Normal game  
420 : Ranked game  
440 : Ranked flex  
400 : ARAM


In [63]:
my_matches

['EUW1_6346829954',
 'EUW1_6346406135',
 'EUW1_6346377480',
 'EUW1_6345492861',
 'EUW1_6344920335',
 'EUW1_6344828393',
 'EUW1_6344780310',
 'EUW1_6340799573',
 'EUW1_6340749762',
 'EUW1_6335975261',
 'EUW1_6335029842',
 'EUW1_6334965764',
 'EUW1_6333590234',
 'EUW1_6333492894',
 'EUW1_6328709921',
 'EUW1_6328662160',
 'EUW1_6328212297',
 'EUW1_6323942872',
 'EUW1_6322470530',
 'EUW1_6322308414']

In [64]:
ranked_list=[]
for games in my_matches:
    match_played=watcher.match.by_id(my_region,games)
    if match_played['info']['queueId']== 420 or match_played['info']['queueId']== 440  :
        ranked_list.append(match_played['metadata']['matchId'])
        
            
    

In [65]:
ranked_list

['EUW1_6344828393', 'EUW1_6344780310', 'EUW1_6333492894']

In [66]:
champ_ranked=[]
for match in ranked_list:
    match_played=watcher.match.by_id(my_region,match)
    for participants in match_played['info']['participants']:
        if participants['summonerName']=="MamaYumYum":
            champ_ranked.append(participants['championName'])
            break

In [67]:
champ_ranked

['Zed', 'Yone', 'Jayce']